In [4]:
'''Trains a QRNN on the IMDB sentiment classification task.
Modified from keras' examples/imbd_lstm.py.
'''
from __future__ import print_function
import numpy as np
import sys
sys.path.append('datasets')

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.datasets import imdb

import codeforces
from qrnn import QRNN

np.random.seed(1337)  # for reproducibility

top_words = 500
max_len = 500  # cut texts after this number of words (among top max_features most common words)
batch_size = 64
embed_dim = 128
qrnn_size = 200
dropout = 0.25
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = codeforces.load_data(path='../data/codeforces_full.pkl',
                                                            num_words=top_words)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(top_words, embed_dim, input_length=max_len))
model.add(QRNN(qrnn_size, window_size=3, dropout=dropout, 
               W_regularizer=l2(1e-4), b_regularizer=l2(1e-4), 
               W_constraint=maxnorm(10), b_constraint=maxnorm(10)))
model.add(Dense(1, activation = 'sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())
print('Train...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_data=(x_test, y_test))

Loading data...
18537 train sequences
4635 test sequences
Pad sequences (samples x time)
x_train shape: (18537, 500)
x_test shape: (4635, 500)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 128)          64000     
_________________________________________________________________
qrnn_4 (QRNN)                (None, 200)               231000    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 295,201.0
Trainable params: 295,201
Non-trainable params: 0.0
_________________________________________________________________
None
Train...
Train on 18537 samples, validate on 4635 samples
Epoch 1/10
18537/18537 [==============================] - 2542s - loss: 0.6878 - acc: 0.5544 - val_loss: 0.6812 - val_acc: 0.5674
Epoch 2/10
18537/18537 [=====================